# Reset the Mean Layers
Our implementation of SchNet does not use atomrefs, but that is equivalent to normalizing by mean given

In [1]:
from fff.learning.gc.ase import SchnetCalculator
from fff.learning.gc.models import load_pretrained_model
from ttm.ase import TTMCalculator
from ase.db import connect
from ase import Atoms, units, build
import numpy as np
import torch

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the isolated atom energies
Used to normalize the energies of SchNetPack models

## Update these values in the network
The atomrefs are stored as an "Embedding" layer

In [2]:
model = load_pretrained_model('../starting-weights/finetune_ttm_alstep_023.pt')

Make a calculator to show how poorly it works

In [3]:
water = build.molecule('H2O')

In [4]:
psi4_eng = TTMCalculator().get_potential_energy(water)

In [5]:
spk_calc = SchnetCalculator(model, 'cpu')
spk_eng = spk_calc.get_potential_energy(water)

In [6]:
print(f'Energy of water - Psi4: {psi4_eng:.2f} eV, SchNet {spk_eng:.2f} eV')

Energy of water - Psi4: 0.01 eV, SchNet -0.02 eV


We're really far off

## Update the mean and standard deviation
These are other features that will adjust for TTM and DFT having different energies per water (0 for TTM, ~-9 eV for others)

In [7]:
dft_energy_per_atom = []
with connect('../../initial-database/initial-ttm.db') as db:
    dft_energy_pa = [a.energy / a.natoms  for a in db.select()]

In [8]:
model.lin2.bias.item()

-0.0064343237318098545

The mean and standard deviation are in the standardization layer

In [9]:
mean = np.mean(dft_energy_pa) - model.lin2.bias.item()  # Average
std = np.std(dft_energy_pa)
print(f'Values with DFT - Mean {mean:.2f}, St. Dev. {std:.3f}')

Values with DFT - Mean -0.09, St. Dev. 0.064


We will now have a ~3 eV offset per atom, which accounts for the difference in bonding

In [10]:
model = load_pretrained_model('../starting-weights/finetune_ttm_alstep_023.pt', mean=mean, std=std)
spk_calc = SchnetCalculator(model, 'cpu')

In [11]:
spk_eng_new = spk_calc.get_potential_energy(water)

In [12]:
print(f'Energy of water - Psi4: {psi4_eng:.2f} eV, SchNet {spk_eng_new:.2f} eV')

Energy of water - Psi4: 0.01 eV, SchNet -0.27 eV


Much closer

## Save Updated Model
For us to use later

In [13]:
torch.save(model, 'starting-model')